# AnalogNAS Tutorial

[AnalogAINAS](https://github.com/IBM/analog-nas) is a framework that aims at building analog-aware efficient deep learning models. AnalogNAS is built on top of the [AIHWKIT](https://github.com/IBM/aihwkit). IBM Analog Hardware Acceleration Kit (AIHWKIT) is an open source Python toolkit for exploring and using the capabilities of in-memory computing devices in the context of artificial intelligence.

In a high-level AnalogAINAS consists of 4 main building blocks which (can) interact with each other:
* Configuration spaces: a search space of architectures targeting a specific dataset.
* Evaluator: a ML predictor model to predict: 
    * 1-day Accuracy: the evaluator models the drift effect that is encountered in Analog devices. The accuracy after 1 day of drift is then predicted and used as an objective to maximize. 
    * The Accuracy Variation for One Month (AVM): The difference between the accuracy after 1 month and the accuracy after 1 sec. 
    * The 1-day accuracy standard deviation: The stochasticity of the noise induces different variation of the model's accuracy depending on its architecture. 
* Optimizer: a optimization strategy such as evolutionary algorithm or bayesian optimization. 
* Worker: A global object that runs the architecture search loop and the final network training pipeline

### Installation and setup
**NOTE:** this installation is tested on a Linux and Windows machine.

Firstly, refer to [AIHWKit installation](https://aihwkit.readthedocs.io/en/latest/install.html) to install Pytorch and the AIHWKit toolkit. 

Install the additional requirements, using:
```
pip install -r requirements.txt 
```

Afterwards, install AnalogNAS by running the ```setup.py``` file:
``` 
python setup.py install 
```

### Create a Configuration Space
AnalogNAS presents a general search space composed of ResNet-like architectures. 

The macro-architecture defined in the file ```search_spaces/resnet_macro_architecture.py``` is customizable to any image classification dataset, given an input shape and output classes. 

In [1]:
from analogainas.search_spaces.config_space import ConfigSpace

In [2]:
## Default Search Space
CS = ConfigSpace('CIFAR-10')
CS

Architecture Type: resnet-like
Search Space Size: 773094113280
------------------------------------------------
0)
Name: out_channel0
Min_Value:0
Max_value:0
Step:1

1)
Name: M
Min_Value:1
Max_value:5
Step:1

2)
Name: R1
Min_Value:1
Max_value:16
Step:1

3)
Name: R2
Min_Value:0
Max_value:16
Step:1

4)
Name: R3
Min_Value:0
Max_value:16
Step:1

5)
Name: R4
Min_Value:0
Max_value:16
Step:1

6)
Name: R5
Min_Value:0
Max_value:16
Step:1

7)
Name: convblock1
Min_Value:0
Max_value:0
Step:1

8)
Name: widenfact1
Min_Value:0.5
Max_value:0.8
Step:1

9)
Name: B1
Min_Value:1
Max_value:5
Step:1

10)
Name: convblock2
Min_Value:0
Max_value:0
Step:1

11)
Name: widenfact2
Min_Value:0.5
Max_value:0.8
Step:1

12)
Name: B2
Min_Value:1
Max_value:5
Step:1

13)
Name: convblock3
Min_Value:0
Max_value:0
Step:1

14)
Name: widenfact3
Min_Value:0.5
Max_value:0.8
Step:1

15)
Name: B3
Min_Value:1
Max_value:5
Step:1

16)
Name: convblock4
Min_Value:0
Max_value:0
Step:1

17)
Name: widenfact4
Min_Value:0.5
Max_value:0.8
St

In [3]:
CS.get_hyperparameters()

['out_channel0', 'M', 'R1', 'R2', 'R3', 'R4', 'R5', 'convblock1', 'widenfact1', 'B1', 'convblock2', 'widenfact2', 'B2', 'convblock3', 'widenfact3', 'B3', 'convblock4', 'widenfact4', 'B4', 'convblock5', 'widenfact5', 'B5']


In [4]:
## Add a hyperparameter 
## Name should be a unique ID. 
CS.add_hyperparameter("out_channel", "discrete", min_value=8, max_value=32, step=3)

In [5]:
## The error is generated on purpose when the same name is given
CS.add_hyperparameter("out_channel", "discrete", min_value=8, max_value=32, step=3)

Exception: Name should be unique!

In [ ]:
CS.compute_cs_size()

6184752906240

In [6]:
CS.remove_hyperparameter("out_channel")
CS.compute_cs_size()

773094113280

In [7]:
# Sample possible configurations
configs = CS.sample_arch_uniformly(5)

In [8]:
# Configs is a dictionary holding 5 possible architectures in our search space. 
configs 

[{'out_channel0': 64,
  'M': 2,
  'R1': 12,
  'R2': 1,
  'R3': 0,
  'R4': 0,
  'R5': 0,
  'convblock1': 2,
  'widenfact1': 0.7898408899988426,
  'B1': 2,
  'convblock2': 2,
  'widenfact2': 0.775246613246175,
  'B2': 1,
  'convblock3': 0,
  'widenfact3': 0,
  'B3': 0,
  'convblock4': 0,
  'widenfact4': 0,
  'B4': 0,
  'convblock5': 0,
  'widenfact5': 0,
  'B5': 0},
 {'out_channel0': 64,
  'M': 2,
  'R1': 8,
  'R2': 0,
  'R3': 0,
  'R4': 0,
  'R5': 0,
  'convblock1': 1,
  'widenfact1': 0.6496111034668584,
  'B1': 1,
  'convblock2': 1,
  'widenfact2': 0.7762654316325677,
  'B2': 4,
  'convblock3': 0,
  'widenfact3': 0,
  'B3': 0,
  'convblock4': 0,
  'widenfact4': 0,
  'B4': 0,
  'convblock5': 0,
  'widenfact5': 0,
  'B5': 0},
 {'out_channel0': 16,
  'M': 4,
  'R1': 13,
  'R2': 5,
  'R3': 11,
  'R4': 5,
  'R5': 0,
  'convblock1': 2,
  'widenfact1': 0.5914426894746183,
  'B1': 4,
  'convblock2': 2,
  'widenfact2': 0.5775186034020032,
  'B2': 1,
  'convblock3': 1,
  'widenfact3': 0.56005672

## Evaluator 

To speed up the search, we built a machine learning predictor to evaluate the accuracy and robustness of any given architecture from the configuration space. 

In [9]:
# Load the evaluator 
from analogainas.evaluators.xgboost import XGBoostEvaluator
evaluator = XGBoostEvaluator()

In [10]:
# The ranker model ranks the architectures according to their 1-day accuracy. It is trained with a listwise training loss. 
evaluator.ranker 

XGBRanker(base_score=0.5, booster='gbtree', callbacks=None, colsample_bylevel=1,
          colsample_bynode=1, colsample_bytree=0.9, early_stopping_rounds=None,
          enable_categorical=True, eta=0.05, eval_metric=None, gamma=0,
          gpu_id=-1, grow_policy='depthwise', importance_type=None,
          interaction_constraints='', learning_rate=0.1, max_bin=256,
          max_cat_to_onehot=4, max_delta_step=0, max_depth=6, max_leaves=0,
          min_child_weight=1, missing=nan, monotone_constraints='()',
          n_estimators=110, n_jobs=0, num_parallel_tree=1, predictor='auto',
          random_state=42, reg_alpha=0, ...)

In [11]:
# The AVM predictor regresses the average monthly variation. 
evaluator.avm_predictor

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.9,
             early_stopping_rounds=None, enable_categorical=False, eta=0.05,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=110, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, ...)

## Search Optimizer and Worker 
In this example, we will use evolutionary search to look for the best architecture in CS using our evaluator. 

In [12]:
from analogainas.search_algorithms.ea_optimized import EAOptimizer
from analogainas.search_algorithms.worker import Worker

c:\Users\hadjer\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
optimizer = EAOptimizer(evaluator, population_size=20, nb_iter=10)  

In [14]:
NB_RUN = 2
worker = Worker(CS, optimizer=optimizer, runs=NB_RUN)

In [16]:
worker.search()

Result directory created.

Search 0 started
ITERATION 0 completed: best acc 0.5867591738425217
ITERATION 1 completed: best acc 0.7953551661210345
ITERATION 2 completed: best acc 0.7953551661210345
ITERATION 3 completed: best acc 0.7953551661210345
ITERATION 4 completed: best acc 0.8079763705587886
ITERATION 5 completed: best acc 0.8337070881276811
ITERATION 6 completed: best acc 0.879439058695786
ITERATION 7 completed: best acc 0.879439058695786
ITERATION 8 completed: best acc 0.879439058695786
ITERATION 9 completed: best acc 0.879439058695786
Best Acc = 0.879439058695786
Search 1 started
ITERATION 0 completed: best acc 0.7832513086778994
ITERATION 1 completed: best acc 0.7832513086778994
ITERATION 2 completed: best acc 0.8130145589367455
ITERATION 3 completed: best acc 0.8130145589367455
ITERATION 4 completed: best acc 0.8531200051528554
ITERATION 5 completed: best acc 0.8531200051528554
ITERATION 6 completed: best acc 0.8531200051528554
ITERATION 7 completed: best acc 0.8624818837214

In [17]:
worker.result_summary()

Best architecture accuracy:  0.879439058695786
Standard deviation of accuracy over 2 runs: 0.879439058695786
